## Create the required objects in Unity Catalog
For this specific DEMO, add the marketplace news data before 

In [0]:
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")
demo_catalog = dbutils.widgets.get("catalog")
demo_schema = dbutils.widgets.get("schema")
print (demo_catalog, demo_schema)

In [ ]:
# Create the catalog, schema and volume
# Create the catalog if it does not exist
#spark.sql(f"CREATE CATALOG IF NOT EXISTS {demo_catalog}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {demo_catalog}.{demo_schema}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {demo_catalog}.{demo_schema}.artifacts")

In [0]:
#Create Assign the permissions to the UC object

spark.sql(f"GRANT ALL PRIVILEGES ON CATALOG {demo_catalog} TO `account users`")
spark.sql(f"GRANT ALL PRIVILEGES ON SCHEMA {demo_catalog}.{demo_schema} TO `account users`")
spark.sql(f"GRANT ALL PRIVILEGES ON VOLUME {demo_catalog}.{demo_schema}.artifacts TO `account users`")

In [0]:
spark.sql(f"""CREATE OR REPLACE FUNCTION {demo_catalog}.{demo_schema}.company_sentiment_articles(
    company STRING COMMENT 'Select a specific company to get a list of articles with sentiment'
  )
  RETURNS TABLE(sentiment STRING, count string)
  COMMENT 'Returns the sentiment of specific news articles a for a specic company'
  RETURN
    select
      sentiment,
      content
    from
      (
        SELECT
          ai_analyze_sentiment(content) AS sentiment,
          content
        FROM
          `news`.`datasets`.`bbc_news`
        WHERE
          headline ILIKE '%' || company || '%'
          OR content ILIKE '%' || company || '%'
          OR topics ILIKE '%' || company || '%'
        union
        SELECT
          ai_analyze_sentiment(content) AS sentiment,
          content
        FROM
          `news`.`datasets`.`cnn_news`
        WHERE
          headline ILIKE '%' || company || '%'
          OR content ILIKE '%' || company || '%'
          OR topics ILIKE '%' || company || '%'
        union
        SELECT
          ai_analyze_sentiment(content) AS sentiment,
          content
        FROM
          `news`.`datasets`.`reuters_news`
        WHERE
          headline ILIKE '%' || company || '%'
          OR content ILIKE '%' || company || '%'
          OR topics ILIKE '%' || company || '%'
      )
    where
      sentiment is not null""")

In [0]:

spark.sql(f"""CREATE OR REPLACE FUNCTION {demo_catalog}.{demo_schema}.company_sentiment_count(
  company STRING COMMENT 'Select a specific company'
)
RETURNS TABLE(sentiment STRING, count BIGINT)
COMMENT 'Returns the sentiment of the market from news article and count per sentiment for a specic company'
RETURN select sentiment, count(*) from (
  SELECT ai_analyze_sentiment(content) AS sentiment, content
  FROM `news`.`datasets`.`bbc_news`
  WHERE headline ILIKE '%' || company || '%' OR content ILIKE '%' || company || '%' OR topics ILIKE '%' || company || '%'
  union
  SELECT ai_analyze_sentiment(content) AS sentiment, content
  FROM `news`.`datasets`.`cnn_news`
  WHERE headline ILIKE '%' || company || '%' OR content ILIKE '%' || company || '%' OR topics ILIKE '%' || company || '%'
  union
  SELECT ai_analyze_sentiment(content) AS sentiment, content
  FROM `news`.`datasets`.`reuters_news`
  WHERE headline ILIKE '%' || company || '%' OR content ILIKE '%' || company || '%' OR topics ILIKE '%' || company || '%'
) where sentiment is not null 
group by sentiment""")

In [0]:
spark.sql(f"""CREATE OR REPLACE FUNCTION {demo_catalog}.{demo_schema}.company_sentiment_percentage_distribution(
  company STRING COMMENT 'Select a specific company to get the percentage distribution'
)
RETURNS TABLE(sentiment STRING, count BIGINT)
COMMENT 'Returns the sentiment percentage distribution of the market from news article for a specic company'
RETURN SELECT
  sentiment,
  ROUND(100.0 * count / SUM(count) OVER (), 2) AS percentage
FROM {demo_catalog}.{demo_schema}.company_sentiment_count(company)""")

In [0]:

spark.sql(f"GRANT EXECUTE ON FUNCTION {demo_catalog}.{demo_schema}.company_sentiment_articles TO `account users`")
spark.sql(f"GRANT EXECUTE ON FUNCTION {demo_catalog}.{demo_schema}.company_sentiment_count TO `account users`")
spark.sql(f"GRANT EXECUTE ON FUNCTION {demo_catalog}.{demo_schema}.company_sentiment_percentage_distribution TO `account users`")

## Create the company_stocks tables


In [0]:
spark.sql(f"""CREATE or REPLACE TABLE {demo_catalog}.{demo_schema}.company_stocks(
  ticker STRING,
  company STRING,
  market STRING,
  stock_amount BIGINT
)""")

spark.sql(f"""INSERT INTO {demo_catalog}.{demo_schema}.company_stocks (ticker, company, market, stock_amount) VALUES
  ('GOOGL', 'Google', 'Technology', 1500),
  ('AAPL', 'Apple', 'Technology', 2000),
  ('SBUX', 'Starbucks', 'Consumer Goods', 500), 
  ('NKE', 'Nike', 'Consumer Goods', 2000), 
  ('TSLA', 'Telsa', 'Consumer Goods', 5000),  
  ('DIS', 'Disney', 'Entertainment', 1000),
  ('NVDA', 'Nvidia', 'Technology', 15000),
  ('AMZN', 'Amazon', 'Consumer Goods', 4000)""")

spark.sql(f"GRANT SELECT ON {demo_catalog}.{demo_schema}.company_stocks TO `account users`")
  

In [0]:
spark.sql(f"GRANT SELECT ON {demo_catalog}.{demo_schema}.company_stocks TO `account users`")